### Import lib

In [1]:
%load_ext lab_black

In [2]:
import os

os.chdir("/cerea_raid/users/dumontj/dev/coco2/dl")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib_functions

matplotlib_functions.setMatplotlibParam()
plt.viridis()
import joblib
import pickle
from include.generators import Generator
from saver import Saver
from include.callbacks import get_modelcheckpoint, ExtraValidation

import model_eval
from Data import Data_train, Data_eval
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import models.reg as rm
from include.loss import rmae
from include.loss import rmae

2023-02-10 18:52:06.145209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 18:52:06.251981: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


<Figure size 1920x1440 with 0 Axes>

In [3]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=45,
    zoom_range=0.2,
)

In [4]:
general_save_dir = "/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/"
project_name = "exp_inputs_ber"

### Data download

In [5]:
path_train_nc = "/libre/dumontj/coco2/dl-input/2km_forBer_2/train_dataset.nc"
path_valid_nc = "/libre/dumontj/coco2/dl-input/2km_forBer_2/valid_dataset.nc"
path_extra_valid_nc = "/libre/dumontj/coco2/dl-input/2km_Ber/valid_dataset.nc"

save_dir = os.path.join(general_save_dir, "ber")

# PLUME

In [6]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume")
data.prepare_output_inversion()

data.x.train.shape (12576, 64, 64, 1)


## EfficientNetB0 - no gen

In [7]:
exp_name = "plume_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


In [8]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [9]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

2023-02-10 18:52:18.018339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 18:52:18.617856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13773 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


### Train 

In [10]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-10 18:52:29.292564: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-10 18:52:31.339636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-10 18:52:31.988151: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-10 18:52:32.172104: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f9481de16b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-10 18:52:32.172137: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-02-10 18:52:32.176192: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer,

392/393 [============================>.] - ETA: 0s - loss: 2.7539
Epoch 1: val_loss improved from inf to 3.63604, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_nogen/w_best.h5
extra_val_loss: 11.02609634399414
393/393 [==============================] - 44s 47ms/step - loss: 2.7525 - val_loss: 3.6360
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 1.9556
Epoch 2: val_loss improved from 3.63604 to 2.19362, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_nogen/w_best.h5
extra_val_loss: 10.8777494430542
393/393 [==============================] - 16s 39ms/step - loss: 1.9556 - val_loss: 2.1936
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 1.8352
Epoch 3: val_loss improved from 2.19362 to 1.88034, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_nogen/w_best.h5
extra_val_loss: 9.899556159973145
393/393 [==============================] - 16s 40

### save

In [12]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [13]:
exp_name = "plume_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▃▁▁▂▄▂▂▂▂▂▁▃▂▃▃▃▄▃▃▃▅▃▃▄▄▄▄▄▅▆▅▄▅▇▅▄▅▄
extra_val_loss,█▆█▅▄▃▁▃▄▄▄▃▄▃▃▂▅▄▃▂▃▂▅▃▄▃▂▄▃▄▂▂▂▄▁▅▅▂▄▂
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.17325
epoch/val_loss,1.60779
extra_val_loss,7.29212


In [14]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [15]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [16]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


2023-02-13 10:19:29.787311: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 3.0634
Epoch 1: val_loss improved from inf to 3.40936, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_gen/w_best.h5
extra_val_loss: 10.568007469177246
393/393 [==============================] - 41s 50ms/step - loss: 3.0634 - val_loss: 3.4094
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 2.6113
Epoch 2: val_loss improved from 3.40936 to 2.58799, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_gen/w_best.h5
extra_val_loss: 9.369988441467285
393/393 [==============================] - 19s 49ms/step - loss: 2.6113 - val_loss: 2.5880
Epoch 3/1000
393/393 [==============================] - ETA: 0s - loss: 2.6452
Epoch 3: val_loss did not improve from 2.58799
extra_val_loss: 10.712000846862793
393/393 [==============================] - 18s 46ms/step - loss: 2.6452 - val_loss: 4.0217
Epoch 4/1000
393/393 [==============================] - ETA:

### save

In [17]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_Eff_gen/scaler.save']

## Linear baseline

In [18]:
exp_name = "plume_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃▂▂▁▂▃▂▁▂▁▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
extra_val_loss,▃▂▄▁▄▅█▆▆▅▆▄▆▅▂▆▆▅▃▆▆▅▄▆▆▅▄▅▅▆▅▃▄▅▅▅▆▅▆▅
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.32362
epoch/val_loss,1.70285
extra_val_loss,9.24722


In [19]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [20]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [21]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
385/393 [============================>.] - ETA: 0s - loss: 2.6185
Epoch 1: val_loss improved from inf to 2.33183, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_linear/w_best.h5
extra_val_loss: 10.143769264221191
393/393 [==============================] - 4s 6ms/step - loss: 2.6146 - val_loss: 2.3318
Epoch 2/200
393/393 [==============================] - ETA: 0s - loss: 2.1903
Epoch 2: val_loss did not improve from 2.33183
extra_val_loss: 10.08775520324707
393/393 [==============================] - 2s 5ms/step - loss: 2.1903 - val_loss: 2.3801
Epoch 3/200
391/393 [============================>.] - ETA: 0s - loss: 2.1464
Epoch 3: val_loss improved from 2.33183 to 2.06000, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_linear/w_best.h5
extra_val_loss: 9.343658447265625
393/393 [==============================] - 2s 6ms/step - loss: 2.1457 - val_loss: 2.0600
Epoch 4/200
392/393 [============================>.] - ETA:

### save

In [22]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_linear/scaler.save']

## Research of a new model

In [23]:
exp_name = "plume_modelX"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▃▁▁▁▁▁▁▂▁▂▂▁▂▂▁▂▂▃▄▂▃▃▃▃▄▂▃▂▃▃▃▃▃▄▄▃▃▄
extra_val_loss,▇▂▂▄▂▅▆▆▅█▄▂▃▆▂▂▄▃▄▄▁▄▁▂▂▄▃▅▂▃▃▁▄▃▄▄▃▂▅▃
epoch/epoch,199
epoch/learning_rate,0.001
epoch/loss,1.22916
epoch/val_loss,2.16744
extra_val_loss,9.52689


In [24]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [25]:
inputs = tf.keras.layers.Input((64, 64, 1))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [26]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 15:15:30.457594: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 2.4003
Epoch 1: val_loss improved from inf to 3.02691, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_modelX/w_best.h5
extra_val_loss: 13.944808006286621
393/393 [==============================] - 8s 15ms/step - loss: 2.4003 - val_loss: 3.0269
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.0787
Epoch 2: val_loss did not improve from 3.02691
extra_val_loss: 14.41879653930664
393/393 [==============================] - 6s 15ms/step - loss: 2.0787 - val_loss: 3.3000
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 1.9247
Epoch 3: val_loss improved from 3.02691 to 1.93251, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_modelX/w_best.h5
extra_val_loss: 12.606639862060547
393/393 [==============================] - 6s 15ms/step - loss: 1.9247 - val_loss: 1.9325
Epoch 4/500
393/393 [==============================] - ETA: 0s - lo

### save

In [27]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_modelX/scaler.save']

# PLUME + PLUME PREC

In [28]:
wandb.finish()

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▄▃▄▂▃▂▂▄▃▂▂▂▂▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▂▁▁▁▂▂▂
extra_val_loss,█▄▆▆▄▅▄▄▄▆▆▃▄▃▅▅▃▂▄▃▃▃▃▄▃▁▃▃▄▂▂▂▂▂▂▂▃▂▃▁
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.73652
epoch/val_loss,1.44059
extra_val_loss,10.22922


In [29]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume", "plume_prec")
data.prepare_output_inversion()

data.x.train.shape (12576, 64, 64, 2)


## EfficientNetB0 - no gen

In [30]:
exp_name = "plume_plumeprec_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [31]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [32]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [33]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-13 16:03:50.876291: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


392/393 [============================>.] - ETA: 0s - loss: 2.6662
Epoch 1: val_loss improved from inf to 4.01373, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_nogen/w_best.h5
extra_val_loss: 11.62227725982666
393/393 [==============================] - 39s 45ms/step - loss: 2.6650 - val_loss: 4.0137
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 1.9586
Epoch 2: val_loss improved from 4.01373 to 2.38842, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_nogen/w_best.h5
extra_val_loss: 10.161368370056152
393/393 [==============================] - 17s 43ms/step - loss: 1.9586 - val_loss: 2.3884
Epoch 3/500
392/393 [============================>.] - ETA: 0s - loss: 1.7999
Epoch 3: val_loss improved from 2.38842 to 2.08406, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_nogen/w_best.h5
extra_val_loss: 10.124606132507324
393/393 [=======

### save

In [34]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [35]:
exp_name = "plume_plumeprec_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▄▆▂▁▄▃▃▅▁▃▂▂▃▅▅▄▃▄▄▄▆▄▄▅▅▃▅▆▅▅▇▅▅▅▆▆▄▄
extra_val_loss,█▅▃▆▄▄▇▃▂▁▃▃▃▃▄▃▄▄▃▅▄▄▃▄▃▃▆▄▆▂▅▄▃▃▅▃▃▃▂▃
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.13862
epoch/val_loss,1.49572
extra_val_loss,7.89832


In [39]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [40]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [41]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/1000


/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (12576, 64, 64, 2) (2 channels).
  warnings.warn(
2023-02-13 18:23:28.178339: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


392/393 [============================>.] - ETA: 0s - loss: 2.9789
Epoch 1: val_loss improved from inf to 3.39492, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_gen/w_best.h5
extra_val_loss: 10.532018661499023
393/393 [==============================] - 41s 50ms/step - loss: 2.9766 - val_loss: 3.3949
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 2.4727
Epoch 2: val_loss improved from 3.39492 to 2.23958, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_gen/w_best.h5
extra_val_loss: 10.335416793823242
393/393 [==============================] - 18s 45ms/step - loss: 2.4727 - val_loss: 2.2396
Epoch 3/1000
393/393 [==============================] - ETA: 0s - loss: 2.4086
Epoch 3: val_loss did not improve from 2.23958
extra_val_loss: 9.59406852722168
393/393 [==============================] - 17s 43ms/step - loss: 2.4086 - val_loss: 2.3506
Epoch 4/1000
393/393 [===================

### save

In [42]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_Eff_gen/scaler.save']

## Linear baseline

In [43]:
exp_name = "plume_plumeprec_linear"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅█▃▄▄▃▄▃▄▂▄▄▃▄▆▄▅▂▂▂▃▂▅▂▂▁▄▃▁▅▃▃▃▂▄▃▃▂▁
extra_val_loss,▄▆▇▆█▅▅▆▆▄▂▂▅▃▃▄▄▃▄▃▅▄▃▃▃▂▃▁▂▃▃▅▂▃▃▂▅▃▂▁
epoch/epoch,999
epoch/learning_rate,0.001
epoch/loss,1.12081
epoch/val_loss,1.67912
extra_val_loss,9.2224


In [44]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [45]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [46]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/200
390/393 [============================>.] - ETA: 0s - loss: 2.5501
Epoch 1: val_loss improved from inf to 2.27070, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_linear/w_best.h5
extra_val_loss: 8.428778648376465
393/393 [==============================] - 4s 7ms/step - loss: 2.5506 - val_loss: 2.2707
Epoch 2/200
386/393 [============================>.] - ETA: 0s - loss: 2.1507
Epoch 2: val_loss did not improve from 2.27070
extra_val_loss: 8.524872779846191
393/393 [==============================] - 3s 7ms/step - loss: 2.1487 - val_loss: 2.4210
Epoch 3/200
384/393 [============================>.] - ETA: 0s - loss: 2.0649
Epoch 3: val_loss improved from 2.27070 to 1.97890, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_linear/w_best.h5
extra_val_loss: 9.57654094696045
393/393 [==============================] - 3s 7ms/step - loss: 2.0639 - val_loss: 1.9789
Epoch 4/200
391/393 [====================

### save

In [47]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_linear/scaler.save']

## Research of a new model

In [48]:
exp_name = "plume_plumeprec_modelX"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃▁▁▁▂▂▂▂▂▂▂▂▃▃▂▂▃▃▂▃▃▃▃▃▄▃▄▃▃▃▃▃▃▄▃▄▄▄▃
extra_val_loss,▁▃▆▄▆▇▅▆▅▅▆▅▆▆▆▇▆▇▆▇▆▇▇▅▆█▆▇▆▇▇▆▇▆▇▇▇▇▆▇
epoch/epoch,199
epoch/learning_rate,0.001
epoch/loss,1.0994
epoch/val_loss,2.06359
extra_val_loss,10.15205


In [49]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [52]:
inputs = tf.keras.layers.Input((64, 64, 2))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [53]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-14 09:53:30.291122: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_6/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


392/393 [============================>.] - ETA: 0s - loss: 2.3379
Epoch 1: val_loss improved from inf to 2.34805, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_modelX/w_best.h5
extra_val_loss: 14.167975425720215
393/393 [==============================] - 8s 16ms/step - loss: 2.3379 - val_loss: 2.3481
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 2.0006
Epoch 2: val_loss improved from 2.34805 to 2.28933, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_modelX/w_best.h5
extra_val_loss: 13.513790130615234
393/393 [==============================] - 6s 15ms/step - loss: 2.0006 - val_loss: 2.2893
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 1.8359
Epoch 3: val_loss did not improve from 2.28933
extra_val_loss: 14.437338829040527
393/393 [==============================] - 6s 15ms/step - loss: 1.8359 - val_loss: 2.6169
Epoch 4/500
393/393 [=========================

### save

In [54]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_modelX/scaler.save']

# PLUME + PLUME PREC + WINDS

In [55]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("plume", "plume_prec", "u_wind", "v_wind")
data.prepare_output_inversion()

data.x.train.shape (12576, 64, 64, 4)


## EfficientNetB0 - no gen

In [56]:
exp_name = "plume_plumeprec_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▄▇▆▄▄▃▅▂▂▂▂▃▄▃▃▂▄▃▃▂▃▃▄▃▃▂▂▂▁▂▃▂▂▃▃▁▂▄▃
extra_val_loss,▁▃▃▆▃▄▄▆▄▅▅▅▅▆▄▆▅▄▃▅▆▇▇▆▃▇█▅▇▆▇█▄▇▆▆▄▇▅▄
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.64621
epoch/val_loss,1.44361
extra_val_loss,12.83939


In [57]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [58]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [59]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

Epoch 1/500


2023-02-14 10:42:59.022216: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_7/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 2.5849
Epoch 1: val_loss improved from inf to 3.73859, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_winds_Eff_nogen/w_best.h5
extra_val_loss: 11.176528930664062
393/393 [==============================] - 41s 47ms/step - loss: 2.5849 - val_loss: 3.7386
Epoch 2/500
393/393 [==============================] - ETA: 0s - loss: 1.8015
Epoch 2: val_loss improved from 3.73859 to 1.90585, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_winds_Eff_nogen/w_best.h5
extra_val_loss: 11.143109321594238
393/393 [==============================] - 17s 42ms/step - loss: 1.8015 - val_loss: 1.9059
Epoch 3/500
393/393 [==============================] - ETA: 0s - loss: 1.6344
Epoch 3: val_loss improved from 1.90585 to 1.69094, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_winds_Eff_nogen/w_best.h5
extra_val_loss: 10.6998643875122

### save

In [60]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

['/cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_winds_Eff_nogen/scaler.save']

## EfficientNetB0 - with gen

In [61]:
exp_name = "plume_plumeprec_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▁▂▂▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▃▂▃▂▃▂▃▂▃▃▂▄▃▃▃▃▃
extra_val_loss,▇█▄▃▂▁▃▄▆▃▅▃▃▄▄▄▃▂▂▂▄▄▂▄▂▂▄▅▃▄▅▂▄▃▄▃▃▂▄▂
epoch/epoch,499
epoch/learning_rate,0.001
epoch/loss,0.1384
epoch/val_loss,1.38536
extra_val_loss,9.73124


In [62]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [63]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

/profils_cerea/dumontj/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

Epoch 1/1000


2023-02-14 12:49:18.717685: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_8/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


393/393 [==============================] - ETA: 0s - loss: 2.8675
Epoch 1: val_loss improved from inf to 4.19594, saving model to /cerea_raid/users/dumontj/dev/coco2/dl/nb_train/sim/ber/plume_plumeprec_winds_Eff_gen/w_best.h5
extra_val_loss: 11.80575942993164
393/393 [==============================] - 43s 52ms/step - loss: 2.8675 - val_loss: 4.1959
Epoch 2/1000
393/393 [==============================] - ETA: 0s - loss: 2.4359
Epoch 2: val_loss did not improve from 4.19594
extra_val_loss: 10.693000793457031
393/393 [==============================] - 18s 47ms/step - loss: 2.4359 - val_loss: 11.0645
Epoch 3/1000
393/393 [==============================] - ETA: 0s - loss: 2.2950
Epoch 3: val_loss did not improve from 4.19594
extra_val_loss: 10.412943840026855
393/393 [==============================] - 19s 47ms/step - loss: 2.2950 - val_loss: 5.8756
Epoch 4/1000
392/393 [============================>.] - ETA: 0s - loss: 2.1767
Epoch 4: val_loss did not improve from 4.19594
extra_val_loss: 11

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "plume_plumeprec_winds_linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Research of a new model

In [ ]:
exp_name = "plume_plumeprec_winds_modelX"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
inputs = tf.keras.layers.Input((64, 64, 4))

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(inputs)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="elu", strides=1)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid", strides=2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)

model = tf.keras.Model([inputs], outputs)

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train,
    data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1,
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

# FIELD NOT NOISY + PERFECT SEG + FIELD PREC + WINDS

In [ ]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input(
    "xco2_noiseless", "weighted_plume", "xco2_noiseless_prec", "u_wind", "v_wind"
)
data.prepare_output_inversion()

## EfficientNetB0 - no gen

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## EfficientNetB0 - with gen

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "xco2noiseless_prec_plume_winds__linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

# FIELD NOISY + PERFECT SEG + FIELD PREC + WINDS

In [ ]:
data = Data_train(path_train_nc, path_valid_nc, path_extra_valid_nc)
data.prepare_input("xco2_noisy", "weighted_plume", "xco2_noisy_prec", "u_wind", "v_wind")
data.prepare_output_inversion()

## EfficientNetB0 - no gen

In [ ]:
exp_name = "xco2_prec_plume_winds_Eff_nogen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=500,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## EfficientNetB0 - with gen

In [ ]:
exp_name = "xco2_prec_plume_winds_Eff_gen"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model_builder = rm.Keras_reg_model_builder("EfficientNetB0", data.x.train.shape[1:], 1)
model = model_builder.get_model()
model.compile("adam", loss=tf.keras.losses.MeanAbsoluteError())

### Train 

In [ ]:
model.fit(
    generator.flow(data.x.train, data.y.train),
    epochs=1000,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Linear baseline

In [ ]:
exp_name = "xco2_prec_plume_winds_linear"
wandb.init(project=project_name, name=exp_name)

In [ ]:
cbs = get_modelcheckpoint(True, [], os.path.join(save_dir, exp_name, "w_best.h5"))
history = ExtraValidation((data.x.extra_valid, data.y.extra_valid))
cbs.append(WandbMetricsLogger())
cbs.append(history)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), input_shape=data.x.fields_input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile("adam", loss="MeanAbsoluteError")

### Train 

In [ ]:
model.fit(
    data.x.train, data.y.train,
    epochs=200,
    callbacks=cbs,
    validation_data=(data.x.valid, data.y.valid),
    verbose=1
)

### save

In [ ]:
model.save(os.path.join(save_dir, exp_name, "w_last.h5"))
joblib.dump(data.x.scaler, os.path.join(save_dir, exp_name, "scaler.save"))

## Eval

In [ ]:
path_eval_nc = path_extra_valid_nc
exp_name = "plume_Eff_nogen"

data = Data_eval(path_eval_nc)
data.prepare_input(model_eval.get_scaler(os.path.join(save_dir, exp_name)), "plume")
data.prepare_output_inversion()
model = model_eval.get_inversion_model(
    os.path.join(save_dir, exp_name), name_w="w_best.h5"
)

In [ ]:
metrics = model_eval.get_inv_metrics_model_on_data(model, data)
print("mae:", np.mean(metrics["mae"]))
print("mape:", np.mean(metrics["mape"]))

In [ ]:
mean_metrics = model_eval.get_inv_mean_loss(data)
print("mae:", np.mean(mean_metrics["mae"]))
print("mape:", np.mean(mean_metrics["mape"]))

In [ ]:
model_eval.get_summary_histo_inversion(model, data)

# FIELD NOT NOISY + GUESS SEG

# FIELD NOISY + GUESS SEG